<a href="https://colab.research.google.com/github/rizkaaa19/thesis/blob/main/tesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

full_df = pd.read_excel("cpdg.xlsx")
full_df['timestamp'] = pd.to_datetime(full_df['timestamp'])

full_df = (
    full_df
    .sort_values('timestamp')
    .drop_duplicates(subset='timestamp')
    .set_index('timestamp')
)

full_df.columns = (
    full_df.columns
    .str.strip()
    .str.replace(' ', '', regex=False)
)

COL_E = 'Easting[m]'
COL_N = 'Northing[m]'
COL_H = 'Ortho.Height[m]'

full_df[COL_E] /= 1000.0
full_df[COL_N] /= 1000.0
full_df[COL_H] /= 1000.0

print("Kolom tersedia:")
print(full_df.columns.tolist())

print("\nPreview data (E, N, H):")
print(full_df[[COL_E, COL_N, COL_H]].head())

Kolom tersedia:
['Easting[m]', 'Northing[m]', 'Ortho.Height[m]']

Preview data (E, N, H):
                      Easting[m]   Northing[m]  Ortho.Height[m]
timestamp                                                      
2021-01-01 06:59:42  6516739.400  9.894526e+07          131.292
2021-01-02 06:59:42  6516737.334  9.894526e+07          132.741
2021-01-03 06:59:42  6516739.724  9.894526e+07          128.494
2021-01-04 06:59:42  6516739.993  9.894526e+07          129.754
2021-01-05 06:59:42  6516738.984  9.894526e+07          128.723


In [2]:
geo_df = full_df[[COL_E, COL_N, COL_H]]

delta_df = geo_df.diff().dropna()
delta_df.columns = ['dE', 'dN', 'dH']

print(delta_df.head())

                        dE     dN     dH
timestamp                               
2021-01-02 06:59:42 -2.066 -2.023  1.449
2021-01-03 06:59:42  2.390  0.318 -4.247
2021-01-04 06:59:42  0.269  2.343  1.260
2021-01-05 06:59:42 -1.009  0.001 -1.031
2021-01-06 06:59:42  1.657 -2.112  1.489


In [3]:
WINDOW_SMOOTH = 7

delta_smooth = delta_df.rolling(
    window=WINDOW_SMOOTH,
    center=False
).mean().dropna()

In [4]:
from sklearn.preprocessing import MinMaxScaler

scaler_E = MinMaxScaler(feature_range=(-1, 1))
scaler_N = MinMaxScaler(feature_range=(-1, 1))
scaler_H = MinMaxScaler(feature_range=(-1, 1))

dE_scaled = scaler_E.fit_transform(delta_smooth[['dE']])
dN_scaled = scaler_N.fit_transform(delta_smooth[['dN']])
dH_scaled = scaler_H.fit_transform(delta_smooth[['dH']])

scaled_data = np.hstack([dE_scaled, dN_scaled, dH_scaled])

In [5]:
TIME_STEPS = 30

def create_sequences(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(data[i+time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_data, TIME_STEPS)

In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Concatenate

input_layer = Input(shape=(TIME_STEPS, 3))

# Branch E
branch_E = Lambda(lambda x: x[:, :, 0:1])(input_layer)
branch_E = LSTM(32, return_sequences=False)(branch_E)

# Branch N
branch_N = Lambda(lambda x: x[:, :, 1:2])(input_layer)
branch_N = LSTM(32, return_sequences=False)(branch_N)

# Branch H
branch_H = Lambda(lambda x: x[:, :, 2:3])(input_layer)
branch_H = LSTM(32, return_sequences=False)(branch_H)

# Gabung
merged = Concatenate()([branch_E, branch_N, branch_H])
dense = Dense(32, activation='relu')(merged)

output = Dense(3, name='Predicted_dENH')(dense)

model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer='adam', loss='mse')

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 30, 3)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 30, 1)     │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 30, 1)     │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_2 (Lambda)   │ (None, 30, 1)     │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 32)        │      4,352 │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 32)        │      4,352 │ lambda_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 32)        │      4,352 │ lambda_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 96)        │          0 │ lstm[0][0],       │
│ (Concatenate)       │                   │            │ lstm_1[0][0],     │
│                     │                   │            │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │      3,104 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Predicted_dENH      │ (None, 3)         │         99 │ dense[0][0]       │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 16,259 (63.51 KB)

 Trainable params: 16,259 (63.51 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
n = len(X)
train_end = int(0.7 * n)
val_end = int(0.85 * n)

X_train, y_train = X[:train_end], y[:train_end]
X_val, y_val = X[train_end:val_end], y[train_end:val_end]
X_test, y_test = X[val_end:], y[val_end:]

In [8]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    verbose=1
)

y_pred = model.predict(X_test)

dE_true = scaler_E.inverse_transform(y_test[:, 0:1])
dN_true = scaler_N.inverse_transform(y_test[:, 1:2])
dH_true = scaler_H.inverse_transform(y_test[:, 2:3])

dE_pred = scaler_E.inverse_transform(y_pred[:, 0:1])
dN_pred = scaler_N.inverse_transform(y_pred[:, 1:2])
dH_pred = scaler_H.inverse_transform(y_pred[:, 2:3])

def metrics(name, y_true, y_hat):
    mse = mean_squared_error(y_true, y_hat)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_hat)
    print(f"\n[{name}]")
    print("MSE :", mse)
    print("RMSE:", rmse)
    print("MAE :", mae)

metrics("dE (m/hari)", dE_true, dE_pred)
metrics("dN (m/hari)", dN_true, dN_pred)
metrics("dH (m/hari)", dH_true, dH_pred)

Epoch 1/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 65ms/step - loss: 0.0246 - val_loss: 0.0142
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0151 - val_loss: 0.0135
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0148 - val_loss: 0.0134
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0137 - val_loss: 0.0133
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0187 - val_loss: 0.0130
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0152 - val_loss: 0.0129
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0159 - val_loss: 0.0129
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0160 - val_loss: 0.0132
Epoch 9/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0137 - val_loss: 0.0131
Epoch 10/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 0.0147 - val_loss: 0.0130
Epoch 11/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0143 - val_loss: 0.0132
Epoch 12/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0

In [9]:
full_df = pd.read_excel("cpdg.xlsx")

full_df['timestamp'] = pd.to_datetime(full_df['timestamp'])

full_df = (
    full_df
    .sort_values('timestamp')
    .drop_duplicates(subset='timestamp')
    .set_index('timestamp')
)

full_df.columns = (
    full_df.columns
    .str.strip()
    .str.replace(' ', '', regex=False)
)

COL_E = 'Easting[m]'
COL_N = 'Northing[m]'
COL_H = 'Ortho.Height[m]'

full_df[COL_E] = full_df[COL_E] / 1000.0
full_df[COL_N] = full_df[COL_N] / 1000.0

full_df[COL_H] = full_df[COL_H] / 1000.0

print("Kolom tersedia:", full_df.columns.tolist())
print("\nPreview data (E, N, H):")
print(full_df[[COL_E, COL_N, COL_H]].head())

Kolom tersedia: ['Easting[m]', 'Northing[m]', 'Ortho.Height[m]']

Preview data (E, N, H):
                      Easting[m]   Northing[m]  Ortho.Height[m]
timestamp                                                      
2021-01-01 06:59:42  6516739.400  9.894526e+07          131.292
2021-01-02 06:59:42  6516737.334  9.894526e+07          132.741
2021-01-03 06:59:42  6516739.724  9.894526e+07          128.494
2021-01-04 06:59:42  6516739.993  9.894526e+07          129.754
2021-01-05 06:59:42  6516738.984  9.894526e+07          128.723


In [10]:
monthly_pos = full_df[[COL_E, COL_N, COL_H]].resample('MS').median()

In [11]:
monthly_delta = monthly_pos.diff().dropna()
monthly_delta.columns = ['dE_m', 'dN_m', 'dH_m']

In [12]:
H_month = monthly_pos[COL_H].copy()

H_trend = H_month.rolling(window=5, center=True, min_periods=1).median()

dH_trend = H_trend.diff().dropna()

monthly_delta2 = monthly_delta.copy()
monthly_delta2['dH_m'] = dH_trend

In [13]:
monthly_delta2['dH_m'] = (
    monthly_delta2['dH_m']
    .rolling(window=3, center=True, min_periods=1)
    .mean()
)

In [14]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

features = monthly_delta2[['dE_m', 'dN_m', 'dH_m']].copy()

scaler_E = MinMaxScaler(feature_range=(-1, 1))
scaler_N = MinMaxScaler(feature_range=(-1, 1))
scaler_H = MinMaxScaler(feature_range=(-1, 1))

dE_scaled = scaler_E.fit_transform(features[['dE_m']])
dN_scaled = scaler_N.fit_transform(features[['dN_m']])
dH_scaled = scaler_H.fit_transform(features[['dH_m']])

scaled_monthly = np.hstack([dE_scaled, dN_scaled, dH_scaled])

In [15]:
import numpy as np

TIME_STEPS = 6

def create_sequences(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(data[i+time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_monthly, TIME_STEPS)

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (41, 6, 3)
y shape: (41, 3)


In [16]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Concatenate

input_layer = Input(shape=(TIME_STEPS, 3), name="Input_6Bulan")

# Branch E
branch_E = Lambda(lambda x: x[:, :, 0:1], name="Select_dE")(input_layer)
branch_E = LSTM(32, return_sequences=False, name="LSTM_dE")(branch_E)

# Branch N
branch_N = Lambda(lambda x: x[:, :, 1:2], name="Select_dN")(input_layer)
branch_N = LSTM(32, return_sequences=False, name="LSTM_dN")(branch_N)

# Branch H
branch_H = Lambda(lambda x: x[:, :, 2:3], name="Select_dH")(input_layer)
branch_H = LSTM(32, return_sequences=False, name="LSTM_dH")(branch_H)

# Gabung semua branch
merged = Concatenate(name="Merge_ENH")([branch_E, branch_N, branch_H])

# Dense
dense = Dense(32, activation="relu", name="Dense_After_Merge")(merged)

# Output: 1 nilai (prediksi dH bulan depan, masih skala -1..1)
output = Dense(1, name="Predicted_dH_next_month")(dense)

# Build & compile
model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer="adam", loss="mse")

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input_6Bulan        │ (None, 6, 3)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Select_dE (Lambda)  │ (None, 6, 1)      │          0 │ Input_6Bulan[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Select_dN (Lambda)  │ (None, 6, 1)      │          0 │ Input_6Bulan[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Select_dH (Lambda)  │ (None, 6, 1)      │          0 │ Input_6Bulan[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ LSTM_dE (LSTM)      │ (None, 32)        │      4,352 │ Select_dE[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ LSTM_dN (LSTM)      │ (None, 32)        │      4,352 │ Select_dN[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ LSTM_dH (LSTM)      │ (None, 32)        │      4,352 │ Select_dH[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Merge_ENH           │ (None, 96)        │          0 │ LSTM_dE[0][0],    │
│ (Concatenate)       │                   │            │ LSTM_dN[0][0],    │
│                     │                   │            │ LSTM_dH[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense_After_Merge   │ (None, 32)        │      3,104 │ Merge_ENH[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Predicted_dH_next_… │ (None, 1)         │         33 │ Dense_After_Merg… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 16,193 (63.25 KB)

 Trainable params: 16,193 (63.25 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
y_dH = y[:, 2:3]

In [18]:
n = len(X)

train_end = int(0.7 * n)
val_end   = int(0.85 * n)

X_train, y_train = X[:train_end], y_dH[:train_end]
X_val,   y_val   = X[train_end:val_end], y_dH[train_end:val_end]
X_test,  y_test  = X[val_end:], y_dH[val_end:]

print("Total sampel:", n)
print("Train:", X_train.shape, y_train.shape)
print("Val  :", X_val.shape, y_val.shape)
print("Test :", X_test.shape, y_test.shape)

Total sampel: 41
Train: (28, 6, 3) (28, 1)
Val  : (6, 6, 3) (6, 1)
Test : (7, 6, 3) (7, 1)


In [19]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=30,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=500,
    batch_size=8,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 239ms/step - loss: 0.2443 - val_loss: 0.4229
Epoch 2/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.2393 - val_loss: 0.3560
Epoch 3/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.1730 - val_loss: 0.3148
Epoch 4/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.2310 - val_loss: 0.2906
Epoch 5/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.2492 - val_loss: 0.2661
Epoch 6/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.1945 - val_loss: 0.2480
Epoch 7/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.2049 - val_loss: 0.2265
Epoch 8/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.1478 - val_loss: 0.2061
Epoch 9/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.2287 - val_loss: 0.1970
Epoch 10/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.1407 - val_loss: 0.1914
Epoch 11/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.1237 - val_loss: 0.1964
Epoch 12/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.1595 - val_l

In [20]:
import numpy as np

y_pred = model.predict(X_test, verbose=0)

dH_true_m = scaler_H.inverse_transform(y_test)
dH_pred_m = scaler_H.inverse_transform(y_pred)

dH_true_mm = dH_true_m * 1000.0
dH_pred_mm = dH_pred_m * 1000.0

In [21]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(dH_true_mm, dH_pred_mm)
rmse = np.sqrt(mse)
mae = mean_absolute_error(dH_true_mm, dH_pred_mm)

In [22]:
result_df = pd.DataFrame({
    "dH_true_mm": dH_true_mm.ravel(),
    "dH_pred_mm": dH_pred_mm.ravel(),
    "error_mm": (dH_pred_mm - dH_true_mm).ravel()
})

In [23]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Concatenate
from tensorflow.keras.callbacks import EarlyStopping

# LOAD DATA
FILE_XLSX = "cpdg.xlsx"
SHEET_NAME = 0
TIME_COL = "timestamp"

COL_E = "Easting [m]"
COL_N = "Northing [m]"
COL_H = "Ortho. Height [m]"

df = pd.read_excel(FILE_XLSX, sheet_name=SHEET_NAME)
df.columns = df.columns.astype(str).str.strip()

if TIME_COL not in df.columns:
    raise ValueError(f"Kolom waktu '{TIME_COL}' tidak ditemukan. Kolom yang ada: {df.columns.tolist()}")

df[TIME_COL] = pd.to_datetime(df[TIME_COL], errors="coerce")
df = df.dropna(subset=[TIME_COL])

for c in [COL_E, COL_N, COL_H]:
    if c not in df.columns:
        raise ValueError(f"Kolom '{c}' tidak ditemukan. Kolom yang ada: {df.columns.tolist()}")

df[COL_E] = pd.to_numeric(df[COL_E], errors="coerce")
df[COL_N] = pd.to_numeric(df[COL_N], errors="coerce")
df[COL_H] = pd.to_numeric(df[COL_H], errors="coerce")
df = df.dropna(subset=[COL_E, COL_N, COL_H])

df = df.sort_values(TIME_COL).drop_duplicates(subset=[TIME_COL]).set_index(TIME_COL)

print("SEBELUM konversi unit:")
print(df[[COL_E, COL_N, COL_H]].head())

# Easting/Northing
if df[COL_E].median() > 1e7:
    df[COL_E] = df[COL_E] / 1000.0
if df[COL_N].median() > 1e7:
    df[COL_N] = df[COL_N] / 1000.0

# Height
if df[COL_H].median() > 1000:
    df[COL_H] = df[COL_H] / 1000.0

print("SESUDAH konversi unit:")
print(df[[COL_E, COL_N, COL_H]].head())
print("\nOK. Data siap:", df.shape)

# SINKRONISASI WAKTU: RESAMPLE BULANAN (median)
monthly_pos = df[[COL_E, COL_N, COL_H]].resample("MS").median()

# TRANSFORMASI KE PERUBAHAN RELATIF: DELTA BULANAN
monthly_delta = monthly_pos.diff().dropna()
monthly_delta.columns = ["dE_m", "dN_m", "dH_m"]  #meter/bulan

# REDUKSI NOISE
H_month = monthly_pos[COL_H].copy()
H_trend = H_month.rolling(window=5, center=True, min_periods=1).median()
dH_trend = H_trend.diff().dropna()

monthly_delta2 = monthly_delta.copy()
monthly_delta2["dH_m"] = dH_trend
monthly_delta2["dH_m"] = monthly_delta2["dH_m"].rolling(window=3, center=True, min_periods=1).mean()

# NORMALISASI PER-BRANCH (E, N, H terpisah)
scaler_E = MinMaxScaler(feature_range=(-1, 1))
scaler_N = MinMaxScaler(feature_range=(-1, 1))
scaler_H = MinMaxScaler(feature_range=(-1, 1))

dE_scaled = scaler_E.fit_transform(monthly_delta2[["dE_m"]])
dN_scaled = scaler_N.fit_transform(monthly_delta2[["dN_m"]])
dH_scaled = scaler_H.fit_transform(monthly_delta2[["dH_m"]])

scaled_monthly = np.hstack([dE_scaled, dN_scaled, dH_scaled])

# WINDOW TIME SERIES (TIME_STEPS = 6 bulan)
TIME_STEPS = 6

def create_sequences(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_monthly, TIME_STEPS)
y_dH = y[:, 2:3]  # target hanya dH
target_months = monthly_delta2.index[TIME_STEPS:]  # timestamp target untuk setiap y

# SPLIT TIME-BASED 70:15:15 (train:val:test)
n = len(X)
train_end = int(0.7 * n)
val_end = int(0.85 * n)

X_train, y_train = X[:train_end], y_dH[:train_end]
X_val, y_val = X[train_end:val_end], y_dH[train_end:val_end]
X_test, y_test = X[val_end:], y_dH[val_end:]
test_months = target_months[val_end:]

print("Train/Val/Test:", X_train.shape, X_val.shape, X_test.shape)


# ARSITEKTUR PARALLEL LSTM (3 cabang: dE, dN, dH)
input_layer = Input(shape=(TIME_STEPS, 3), name="Input_6Bulan")

branch_E = Lambda(lambda x: x[:, :, 0:1], name="Select_dE")(input_layer)
branch_E = LSTM(32, return_sequences=False, name="LSTM_dE")(branch_E)

branch_N = Lambda(lambda x: x[:, :, 1:2], name="Select_dN")(input_layer)
branch_N = LSTM(32, return_sequences=False, name="LSTM_dN")(branch_N)

branch_H = Lambda(lambda x: x[:, :, 2:3], name="Select_dH")(input_layer)
branch_H = LSTM(32, return_sequences=False, name="LSTM_dH")(branch_H)

merged = Concatenate(name="Merge_ENH")([branch_E, branch_N, branch_H])
dense = Dense(32, activation="relu", name="Dense_After_Merge")(merged)
output = Dense(1, name="Predicted_dH_next_month")(dense)

model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer="adam", loss=tf.keras.losses.Huber(delta=0.2))
model.summary()

# TRAIN
early_stop = EarlyStopping(monitor="val_loss", patience=30, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=500,
    batch_size=8,
    callbacks=[early_stop],
    verbose=1
)


y_pred_test = model.predict(X_test, verbose=0)

dH_true_m_test = scaler_H.inverse_transform(y_test).ravel()
dH_pred_m_test = scaler_H.inverse_transform(y_pred_test).ravel()

mse = mean_squared_error(dH_true_m_test, dH_pred_m_test)
rmse = np.sqrt(mse)
mae = mean_absolute_error(dH_true_m_test, dH_pred_m_test)

eval_table = pd.DataFrame({
    "Model": ["PLSTM"],
    "MSE": [mse],
    "RMSE": [rmse],
    "MAE": [mae]
})

print("\n=== TABEL EVALUASI ===")
print(eval_table)

# TABEL TEST 2024
first_test_month = pd.Timestamp(test_months[0])
baseline_month = first_test_month - pd.offsets.MonthBegin(1)

if baseline_month in monthly_pos.index:
    H0_abs = float(monthly_pos.loc[baseline_month, COL_H])
else:
    prev_idx = monthly_pos.index[monthly_pos.index < baseline_month]
    if len(prev_idx) == 0:
        raise ValueError("Tidak ditemukan baseline H sebelum data test.")
    H0_abs = float(monthly_pos.loc[prev_idx[-1], COL_H])

H_obs_abs = H0_abs + np.cumsum(dH_true_m_test)
H_pred_abs = H0_abs + np.cumsum(dH_pred_m_test)

subs_obs_m = -(H_obs_abs - H0_abs)
subs_pred_m = -(H_pred_abs - H0_abs)

amp_test = np.nanmax(subs_obs_m) - np.nanmin(subs_obs_m)
scale_m_test = 1.0 if amp_test == 0 else (amp_test / 1.0)

obs_series = 2.0 + (subs_obs_m / scale_m_test)
pred_series = 2.0 + (subs_pred_m / scale_m_test)
label_unit = "SubsidenE"

result_level_test = pd.DataFrame({
    "timestamp": test_months,
    "Observation": obs_series,
    "Prediction": pred_series
}).set_index("timestamp")

result_level_test["Year"] = result_level_test.index.year
result_level_test["Month"] = result_level_test.index.strftime("%B")

obs_pivot = result_level_test.pivot_table(index="Month", columns="Year", values="Observation", aggfunc="mean")
pred_pivot = result_level_test.pivot_table(index="Month", columns="Year", values="Prediction", aggfunc="mean")

month_order = ["January","February","March","April","May","June",
               "July","August","September","October","November","December"]
obs_pivot = obs_pivot.reindex(month_order)
pred_pivot = pred_pivot.reindex(month_order)

months_to_keep = ["June","July","August","September","October","November","December"]
obs_pivot = obs_pivot.loc[months_to_keep]
pred_pivot = pred_pivot.loc[months_to_keep]

obs_pivot.loc["Total"] = obs_pivot.sum(axis=0, numeric_only=True)
pred_pivot.loc["Total"] = pred_pivot.sum(axis=0, numeric_only=True)

print(f"\n=== OBSERVATION TEST ({label_unit}) ===")
print(obs_pivot.round(2))

print(f"\n=== PREDICTION TEST ({label_unit}) ===")
print(pred_pivot.round(2))


H_hist = monthly_pos[COL_H].copy()
subs_hist_m = -(H_hist - H_hist.iloc[0])

seasonal = subs_hist_m.groupby(subs_hist_m.index.month).mean()
seasonal = seasonal - seasonal.mean()  # zero-mean

if seasonal.abs().max() == 0:
    seasonal_pattern = seasonal * 0.0
else:
    seasonal_pattern = seasonal / seasonal.abs().max() * 0.12  # amplitudo 0.12 (index units)

last_known_month_in_scaled_data = monthly_delta2.index[-1]

current_input_sequence = scaled_monthly[-TIME_STEPS:]

num_forecast_months = 12  # Forecast for the next 12 months

future_months = pd.date_range(
    start=last_known_month_in_scaled_data + pd.offsets.MonthBegin(1),
    periods=num_forecast_months,
    freq='MS'
)

# forecasted dH (meters/month)
dH_pred_m_forecast = []

for _ in range(num_forecast_months):
    input_for_prediction = current_input_sequence.reshape(1, TIME_STEPS, 3)

    dH_scaled_pred_next = model.predict(input_for_prediction, verbose=0)

    dH_m_pred_next = scaler_H.inverse_transform(dH_scaled_pred_next).flatten()[0]
    dH_pred_m_forecast.append(dH_m_pred_next)

    last_dE_scaled = current_input_sequence[-1, 0]
    last_dN_scaled = current_input_sequence[-1, 1]

    new_time_step = np.array([[last_dE_scaled, last_dN_scaled, dH_scaled_pred_next.flatten()[0]]])

    current_input_sequence = np.vstack([current_input_sequence[1:], new_time_step])

dH_pred_m_forecast = np.array(dH_pred_m_forecast)

H_start_forecast = monthly_pos[COL_H].iloc[-1]
H_pred_abs_forecast = H_start_forecast + np.cumsum(dH_pred_m_forecast)

subs_future_m = -(H_pred_abs_forecast - H_start_forecast)

# FORECAST
forecast_scaled_list = []
for ts, subs_m in zip(future_months, subs_future_m):
    seas = float(seasonal_pattern.loc[ts.month])
    val = 2.0 + (subs_m / scale_m_test) + seas
    forecast_scaled_list.append(val)

forecast_scaled_df = pd.DataFrame({
    "timestamp": future_months,
    "Forecast_Scaled_2_3": forecast_scaled_list
}).set_index("timestamp")

forecast_scaled_df["Year"] = forecast_scaled_df.index.year
forecast_scaled_df["Month"] = forecast_scaled_df.index.strftime("%B")

forecast_scaled_pivot = forecast_scaled_df.pivot_table(
    index="Month",
    columns="Year",
    values="Forecast_Scaled_2_3",
    aggfunc="mean"
)

forecast_scaled_pivot = forecast_scaled_pivot.reindex(month_order)
forecast_scaled_pivot.loc["Total"] = forecast_scaled_pivot.sum(axis=0, numeric_only=True)

print("\n=== FORECAST ===")
print(forecast_scaled_pivot.round(2))


SEBELUM konversi unit:
                     Easting [m]  Northing [m]  Ortho. Height [m]
timestamp                                                        
2021-01-01 06:59:42   6516739400   98945262132             131292
2021-01-02 06:59:42   6516737334   98945260109             132741
2021-01-03 06:59:42   6516739724   98945260427             128494
2021-01-04 06:59:42   6516739993   98945262770             129754
2021-01-05 06:59:42   6516738984   98945262771             128723
SESUDAH konversi unit:
                     Easting [m]  Northing [m]  Ortho. Height [m]
timestamp                                                        
2021-01-01 06:59:42  6516739.400  9.894526e+07            131.292
2021-01-02 06:59:42  6516737.334  9.894526e+07            132.741
2021-01-03 06:59:42  6516739.724  9.894526e+07            128.494
2021-01-04 06:59:42  6516739.993  9.894526e+07            129.754
2021-01-05 06:59:42  6516738.984  9.894526e+07            128.723

OK. Data siap: (1450, 3)
Trai

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input_6Bulan        │ (None, 6, 3)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Select_dE (Lambda)  │ (None, 6, 1)      │          0 │ Input_6Bulan[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Select_dN (Lambda)  │ (None, 6, 1)      │          0 │ Input_6Bulan[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Select_dH (Lambda)  │ (None, 6, 1)      │          0 │ Input_6Bulan[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ LSTM_dE (LSTM)      │ (None, 32)        │      4,352 │ Select_dE[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ LSTM_dN (LSTM)      │ (None, 32)        │      4,352 │ Select_dN[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ LSTM_dH (LSTM)      │ (None, 32)        │      4,352 │ Select_dH[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Merge_ENH           │ (None, 96)        │          0 │ LSTM_dE[0][0],    │
│ (Concatenate)       │                   │            │ LSTM_dN[0][0],    │
│                     │                   │            │ LSTM_dH[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense_After_Merge   │ (None, 32)        │      3,104 │ Merge_ENH[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Predicted_dH_next_… │ (None, 1)         │         33 │ Dense_After_Merg… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 16,193 (63.25 KB)

 Trainable params: 16,193 (63.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 249ms/step - loss: 0.0551 - val_loss: 0.0770
Epoch 2/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0463 - val_loss: 0.0607
Epoch 3/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0463 - val_loss: 0.0514
Epoch 4/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0479 - val_loss: 0.0469
Epoch 5/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0547 - val_loss: 0.0463
Epoch 6/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0475 - val_loss: 0.0495
Epoch 7/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0372 - val_loss: 0.0549
Epoch 8/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0479 - val_loss: 0.0609
Epoch 9/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0424 - val_loss: 0.0652
Epoch 10/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0466 - val_loss: 0.0663
Epoch 11/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0449 - val_loss: 0.0642
Epoch 12/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0351 - val_l


=== FORECAST ===
Year        2025
Month           
January     2.06
February    2.15
March       2.26
April       2.40
May         2.57
June        2.74
July        2.94
August      3.25
September   3.29
October     3.47
November    3.71
December    3.93
Total      34.78


In [24]:
!pwd
!ls

/content
cpdg.xlsx  sample_data


In [25]:
!mkdir -p deploy
!ls

cpdg.xlsx  deploy  sample_data


In [26]:
%cd /content/deploy
!pwd
!ls

/content/deploy
/content/deploy


In [27]:
%%writefile app.py
import streamlit as st

st.set_page_config(page_title="Demo PLSTM", layout="centered")
st.title("PLSTM Subsidence App (Dummy)")
st.write("app.py berhasil dijalankan.")

Writing app.py


In [28]:
!ls

app.py


In [29]:
%%writefile requirements.txt
streamlit
pandas
numpy
scikit-learn
tensorflow
openpyxl
joblib

Writing requirements.txt


In [30]:
!ls

app.py	requirements.txt


In [31]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.0 MB/s eta 0:00:00


In [34]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.194.187.61:8501

  Stopping...
^C


In [35]:
%%writefile app.py

import streamlit as st
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Concatenate
from tensorflow.keras.callbacks import EarlyStopping

# =========================================================
# STREAMLIT UI
# =========================================================
st.set_page_config(page_title="PLSTM Land Subsidence", layout="centered")
st.title("AI-Based Land Subsidence Prediction (PLSTM)")
st.write("Upload data GNSS (Excel), lalu jalankan model.")

uploaded_file = st.file_uploader("Upload file Excel (.xlsx)", type=["xlsx"])

if uploaded_file is None:
    st.info("Silakan upload file Excel untuk memulai.")
    st.stop()

# LOAD DATA
TIME_COL = "timestamp"
COL_E = "Easting [m]"
COL_N = "Northing [m]"
COL_H = "Ortho. Height [m]"

df = pd.read_excel(uploaded_file)
df.columns = df.columns.astype(str).str.strip()

df[TIME_COL] = pd.to_datetime(df[TIME_COL], errors="coerce")
df = df.dropna(subset=[TIME_COL])

df[COL_E] = pd.to_numeric(df[COL_E], errors="coerce")
df[COL_N] = pd.to_numeric(df[COL_N], errors="coerce")
df[COL_H] = pd.to_numeric(df[COL_H], errors="coerce")
df = df.dropna(subset=[COL_E, COL_N, COL_H])

df = df.sort_values(TIME_COL).set_index(TIME_COL)

# Fix unit
if df[COL_E].median() > 1e7:
    df[COL_E] /= 1000.0
if df[COL_N].median() > 1e7:
    df[COL_N] /= 1000.0
if df[COL_H].median() > 1000:
    df[COL_H] /= 1000.0

st.success("Data berhasil dibaca dan divalidasi.")

monthly_pos = df[[COL_E, COL_N, COL_H]].resample("MS").median()

monthly_delta = monthly_pos.diff().dropna()
monthly_delta.columns = ["dE_m", "dN_m", "dH_m"]

# REDUKSI NOISE
H_trend = monthly_pos[COL_H].rolling(5, center=True, min_periods=1).median()
monthly_delta["dH_m"] = H_trend.diff().dropna()
monthly_delta["dH_m"] = monthly_delta["dH_m"].rolling(3, center=True, min_periods=1).mean()

# NORMALISASI
scaler_E = MinMaxScaler((-1, 1))
scaler_N = MinMaxScaler((-1, 1))
scaler_H = MinMaxScaler((-1, 1))

dE = scaler_E.fit_transform(monthly_delta[["dE_m"]])
dN = scaler_N.fit_transform(monthly_delta[["dN_m"]])
dH = scaler_H.fit_transform(monthly_delta[["dH_m"]])

scaled = np.hstack([dE, dN, dH])

# WINDOWING
TIME_STEPS = 6

def make_seq(data, ts):
    X, y = [], []
    for i in range(len(data) - ts):
        X.append(data[i:i+ts])
        y.append(data[i+ts, 2])
    return np.array(X), np.array(y)

X, y = make_seq(scaled, TIME_STEPS)
y = y.reshape(-1, 1)

# SPLIT DATA
n = len(X)
train_end = int(0.7 * n)
val_end = int(0.85 * n)

X_train, y_train = X[:train_end], y[:train_end]
X_val, y_val = X[train_end:val_end], y[train_end:val_end]
X_test, y_test = X[val_end:], y[val_end:]

test_months = monthly_delta.index[TIME_STEPS + val_end:]

# PARALLEL LSTM
inp = Input(shape=(TIME_STEPS, 3))

bE = LSTM(32)(Lambda(lambda x: x[:, :, 0:1])(inp))
bN = LSTM(32)(Lambda(lambda x: x[:, :, 1:2])(inp))
bH = LSTM(32)(Lambda(lambda x: x[:, :, 2:3])(inp))

x = Concatenate()([bE, bN, bH])
x = Dense(32, activation="relu")(x)
out = Dense(1)(x)

model = Model(inp, out)
model.compile(optimizer="adam", loss=tf.keras.losses.Huber(0.2))

# TRAIN
with st.spinner("Training PLSTM..."):
    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=300,
        batch_size=8,
        callbacks=[EarlyStopping(patience=30, restore_best_weights=True)],
        verbose=0
    )

st.success("Training selesai.")

# EVALUASI 2024 (TEST)
y_pred = model.predict(X_test, verbose=0)

y_true_m = scaler_H.inverse_transform(y_test)
y_pred_m = scaler_H.inverse_transform(y_pred)

mse = mean_squared_error(y_true_m, y_pred_m)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true_m, y_pred_m)

st.subheader("Evaluasi Test (2024)")
st.write(pd.DataFrame({
    "MSE": [mse],
    "RMSE": [rmse],
    "MAE": [mae]
}))

# FORECAST 2025–2029
last_seq = X[-1]
future = []
cur = last_seq.copy()

for _ in range(60):
    p = model.predict(cur.reshape(1, TIME_STEPS, 3), verbose=0)[0, 0]
    future.append(p)
    cur = np.roll(cur, -1, axis=0)
    cur[-1, 2] = p

future_m = scaler_H.inverse_transform(np.array(future).reshape(-1, 1)).ravel()

future_months = pd.date_range(start=monthly_delta.index[-1] + pd.DateOffset(months=1),
                              periods=60, freq="MS")

forecast_df = pd.DataFrame({
    "Predicted_dH_m": future_m,
    "Predicted_dH_mm": future_m * 1000
}, index=future_months)

st.subheader("Forecast dH 2025–2029")
st.dataframe(forecast_df.head(12))

Overwriting app.py


In [36]:
!ls

app.py	requirements.txt


In [37]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.194.187.61:8501

  Stopping...
Traceback (most recent call last):
  File "/usr/local/bin/streamlit", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1485, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1406, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1873, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1269, in invoke
    return ctx.invoke(self.callback, **ctx.params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^